In [60]:
import pytz
from datetime import datetime
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
from requests.adapters import HTTPAdapter, Retry
import time

requests.adapters.DEFAULT_POOLSIZE = 100
retries = Retry(total=20)
# 세션을 생성하고 ConnectionPool 크기를 설정
session = requests.Session()
session.mount('https://', requests.adapters.HTTPAdapter(pool_connections=100, pool_maxsize=100, max_retries=retries))


In [69]:
def get_items(detail_soup):
    addr = detail_soup.find('div', class_="m-exhibitor-entry__item__body__contacts__address")
    website = detail_soup.find('div', class_="m-exhibitor-entry__item__body__contacts__additional__website")
    sku_list = detail_soup.find_all('li', class_='m-exhibitor-entry__item__header__categories__item')

    address = ''
    if addr:
        if addr.find('h4').next_sibling:
            address += addr.find('h4').next_sibling.strip().replace('\n', '  ')
        if addr.find('br') and addr.find('br').next_sibling:
            address += ' '
            address += addr.find('br').next_sibling.strip().replace('\n', '  ')

    if not sku_list:
        sku_list = []

    item = {
        "address": address,
        'website': website.find('a')['href'] if website and website.find('a') else '',
        'sku': [sku.text.strip() for sku in sku_list if sku and sku.text]
    }
    return item


def crawling_comp_list(url):
    response = session.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    a_list = soup.find_all('a', class_='m-exhibitors-list__items__item__name__link js-librarylink-entry')

    result = []
    for i, a in enumerate(a_list):
        prefix = "https://www.gulfood.com/"
        target = a['href'].split("'")[1]

        time.sleep(0.3)        
        detail_response = session.get(prefix + target)
        detail_soup = BeautifulSoup(detail_response.text, 'html.parser')

        comp = get_items(detail_soup)
        comp['name'] = a.text.strip()
        result.append(comp)

    return result
        

#result = crawling_comp_list('https://www.gulfood.com/exhibitors?categories=66AAF122-0B0C-4887-F843EE8C3C8DBDAB&searchgroup=D1CFEFEE-exhibitorslist-2024')
result2 = crawling_comp_list('https://www.gulfood.com/exhibitors?categories=66ACC063-EF15-B1AF-472DE99BF6526B85&searchgroup=D1CFEFEE-exhibitorslist-2024')


In [72]:
import pandas as pd

result_list = []
result_list.extend(result)
result_list.extend(result2)


In [74]:

df = pd.DataFrame(result_list)
df['sku'] = df['sku'].apply(lambda x: ', '.join(x))


In [77]:
df.to_csv('./output/gulfood_20240123_1318.csv', index=False)